In [14]:
import pandas as pd
import pysal
import numpy as np
from esda.moran import Moran
from libpysal import weights

In [2]:
combined_df_no_nans = pd.read_parquet("/cluster/home/maikents/sinmod_features_masters/dataframes_pipeline_1/total_df_no_nans_and_labels.parquet")

In [4]:
coral_df = combined_df_no_nans[['x', 'y', 'labels']]

In [9]:
absence_df = coral_df[coral_df['labels'] == 0]
downsampled_absences = absence_df.sample(n=828, random_state=42)

In [10]:
presence_df = coral_df[coral_df['labels'] == 1]

balanced_df = pd.concat([downsampled_absences, presence_df])

balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [11]:
labels = balanced_df['labels'].values.reshape(-1, 1)

In [12]:
coords = balanced_df[['x', 'y']].values

In [15]:
w = weights.KNN.from_array(coords, k=4)  # Create a KNN spatial weights matrix based on the coords
                                         # k=4 means that the matrix should include the 4 nearest
                                         # neighbors for each data point.

/cluster/home/maikents/PyEnvCoralMapping/lib/python3.12/site-packages/libpysal/weights/distance.py:153: UserWarning: The weights matrix is not fully connected: 
 There are 9 disconnected components.
  W.__init__(self, neighbors, id_order=ids, **kwargs)


In [16]:
w.transform = 'r'

In [17]:
moran = Moran(labels, w)

print(f"Moran's I: {moran.I}")
print(f"p-value: {moran.p_sim}")

Moran's I: 0.7185990338164251
p-value: 0.001
